# import modules

In [1]:
!pip install -q -U json-lines

import json
import os
import pandas as pd
import tarfile
import json_lines
from tqdm.notebook import tqdm
from pprint import pprint
import glob
import matplotlib.pyplot as plt
import numpy as np

# read party, tweets, mentions and hashtags

In [2]:
files123 = glob.glob('./data/*.jl')

partyname_n_tweets = []
partyname_mentions = []
partyname_hash = []
error = []

i = 0

for file in tqdm(files123): # responsible for visual output

    #if(i == 300):
        #continue
    #i += 1
    
    with json_lines.open(file) as f:
        for item in f:
            try:
                if item['response']['meta']['result_count'] == 0: 
                    continue
                elif item['account_data']['Partei'] == "Fraktionslos":
                    continue
                if item['http_status'] == 200:
                    
                    #get party name
                    partei = item["account_data"]["Partei"]
                    
                    #get all tweets per person
                    tweets_array = item["response"]["data"]
                    
                    for tweet in tweets_array: #range(0,len(tweets_array)-1):
                       # print(type(tweet))
                        if("referenced_tweets" in tweet): # tweet is a dictionary
                            tweet_type = tweet["referenced_tweets"]
                            if(tweet_type[0]["type"] == "retweeted"):
                                continue
                        partyname_n_tweets.append({"Partei": partei, "Tweet": tweet["text"]}) # hashtags, mentions in eigene Arrays
                        
                        #if("entities" in tweet): 
                            #entities = tweet["entities"]
                            #if("mentions" in entities):
                                #mentions = entities["mentions"]
                                #for ment in mentions:
                                        #mention = ment["username"]
                                        #partyname_mentions.append({"Partei": partei, "mentions": mention})
                            #if("hashtags" in entities):
                                #hashtags = entities["hashtags"]
                                #for tag in hashtags:
                                        #hashtag = tag["tag"]
                                        #partyname_hash.append({"Partei": partei, "hashtags": hashtag})
                                        
                else:
                    error.append(item)
                    print("yikes")
                    
            except Exception as e:
                raise e
                           
print("Number of error-requests:", len(error))
print("Read", len(partyname_n_tweets), "Tweets sucussfully.")
print("mentions", len(partyname_mentions))
print("hashtags", len(partyname_hash))


Number of error-requests: 0
Read 1806094 Tweets sucussfully.
mentions 0
hashtags 0


# import nltk and sklearn

In [5]:
#weitere imports
import re
import nltk
import string
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split

#import ML LIB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

#import Evaluation Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [6]:
#get stemmer, stop_words
stemmer = nltk.stem.SnowballStemmer("german")
nltk.download("stopwords")
stop_words=set(nltk.corpus.stopwords.words("german"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Runjie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
alles_df = pd.DataFrame(partyname_n_tweets)
alles_df

,Partei,Tweet
0,Bündnis 90/Die Grünen,@green_clarity99 Danke dir!
1,Bündnis 90/Die Grünen,Die Industrie ist das Rückgrat unseres Wohlsta...
2,Bündnis 90/Die Grünen,@peter_simone @RenateKuenast Herzlichen Dank l...
3,Bündnis 90/Die Grünen,"Ebenfalls herzlichen Glückwunsch, liebe @Renat..."
4,Bündnis 90/Die Grünen,@BernhardPoetter treffend: „🌍 CO2-Rückgang 202...
...,...,...
1806089,CDU,Mit bis zu 20 Mio € wird das ⁦@BMBF_Bund⁩ künf...
1806090,CDU,Die Simulation ist neben Theorie und Experimen...
1806091,CDU,Schule darf keine „digitalen Analphabeten“ erz...
1806092,CDU,@StefanKaufmann @helmholtz_de 👍


In [7]:
alles_df["Partei"].value_counts()

Bündnis 90/Die Grünen    433846
SPD                      354097
CDU                      336243
Die Linke                326787
FDP                      182542
AfD                      102382
CSU                       70197
Name: Partei, dtype: int64

In [8]:
len(alles_df)

1806094

In [10]:
import re
import string

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop_words]
    return " ".join(filtered_words)

In [11]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
stop_words

{'aber',
 'alle',
 'allem',
 'allen',
 'aller',
 'alles',
 'als',
 'also',
 'am',
 'an',
 'ander',
 'andere',
 'anderem',
 'anderen',
 'anderer',
 'anderes',
 'anderm',
 'andern',
 'anderr',
 'anders',
 'auch',
 'auf',
 'aus',
 'bei',
 'bin',
 'bis',
 'bist',
 'da',
 'damit',
 'dann',
 'das',
 'dass',
 'dasselbe',
 'dazu',
 'daß',
 'dein',
 'deine',
 'deinem',
 'deinen',
 'deiner',
 'deines',
 'dem',
 'demselben',
 'den',
 'denn',
 'denselben',
 'der',
 'derer',
 'derselbe',
 'derselben',
 'des',
 'desselben',
 'dessen',
 'dich',
 'die',
 'dies',
 'diese',
 'dieselbe',
 'dieselben',
 'diesem',
 'diesen',
 'dieser',
 'dieses',
 'dir',
 'doch',
 'dort',
 'du',
 'durch',
 'ein',
 'eine',
 'einem',
 'einen',
 'einer',
 'eines',
 'einig',
 'einige',
 'einigem',
 'einigen',
 'einiger',
 'einiges',
 'einmal',
 'er',
 'es',
 'etwas',
 'euch',
 'euer',
 'eure',
 'eurem',
 'euren',
 'eurer',
 'eures',
 'für',
 'gegen',
 'gewesen',
 'hab',
 'habe',
 'haben',
 'hat',
 'hatte',
 'hatten',
 'hier',


In [13]:
alles_df["Tweet"] = alles_df.Tweet.map(remove_URL) #map(lambdax: remove_URL(x))
alles_df["Tweet"] = alles_df.Tweet.map(remove_punct)
alles_df["Tweet"] = alles_df.Tweet.map(remove_stopwords)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(alles_df["Tweet"], alles_df.Partei, test_size = 0.3, random_state=111)

# SVM

In [29]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipeline = Pipeline([("vect", TfidfVectorizer(ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=50000)),
                     ("clf", LinearSVC(C=1.0, penalty="l1", max_iter=6000, dual=False))])

In [30]:
model = pipeline.fit(X_train, y_train)

In [31]:
vectorizer = model.named_steps["vect"]
chi = model.named_steps["chi"]
clf = model.named_steps["clf"]

In [32]:
feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices = True)]
feature_names = np.asarray(feature_names)

In [33]:
#target_names = ["Bündnis 90/Die Grünen", "SPD", "CDU", "Die Linke", "FDP", "AfD", "CSU"]
target_names = clf.classes_
print("top 30 keywords per class:")
for i, label in enumerate(target_names):
    top30 = np.argsort(clf.coef_[i])[-30:]
    print("%s: %s" % (label, " ".join(feature_names[top30])))
    print("\n")

top 30 keywords per class:
AfD: grünixchen socialismkills repswalwell bessermenschen merkelland2018 altparteienstaat lockdownkrise brandnerunterwegs syrienreise niewiedersozialismus hollnagel gezabschaffen altparteien staatsfunk kestner klimawahn intelligenzgibtsnichtimbioladen nurdieafd toterjournalismus afdsthueringen teammünzenmaier afd17 traudichdeutschland altparteienstaat2019 buntland ramelüringen afdimbundestag afdwählen nurnochafd afdwirkt


Bündnis 90/Die Grünen: aliabunimah marclueger lobbyistengaben enveryucel به erststimmemutlu israel09 no2isis ursulakramer12 از stefanpoloczek weareinthistogether mehrmainz lkosnabrueck parteispendenaffäre buergerschafthst bdk11 greencongress patientnnen hlk ditibkoeln selcukrsirin greenrunners no2hate ssr kreistagvr zukunftentscheidetsichhier mutludirekt prayforthem bü


CDU: 2rg lsauerwald international order rheinwied fgiarra66 cducsubt get ostprignitzruppin kolumne welt tigarus86 windhagen neersen fightcancelculture grossjenny kgedialog 

In [34]:
prediction = model.predict(X_test)

In [35]:
print("******SVM classification report (fulltext, all data)******")
print("accuracy:", accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction, target_names=target_names, digits=4))

******SVM classification report (fulltext, all data)******
accuracy: 0.588148659447907
                       precision    recall  f1-score   support

                  AfD     0.6765    0.6159    0.6448     30818
Bündnis 90/Die Grünen     0.5374    0.6654    0.5946    130070
                  CDU     0.5613    0.5732    0.5672    100514
                  CSU     0.6779    0.3698    0.4786     21044
            Die Linke     0.6289    0.6250    0.6269     98403
                  FDP     0.7154    0.5066    0.5932     54510
                  SPD     0.5738    0.5506    0.5620    106470

             accuracy                         0.5881    541829
            macro avg     0.6244    0.5581    0.5810    541829
         weighted avg     0.5969    0.5881    0.5872    541829



In [36]:
print("************************SVM confusion matrix (fulltext, all data)************************")
cm = pd.DataFrame(
    confusion_matrix(y_test, prediction, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

************************SVM confusion matrix (fulltext, all data)************************


,pred:AfD,pred:Bündnis 90/Die Grünen,pred:CDU,pred:CSU,pred:Die Linke,pred:FDP,pred:SPD
true:AfD,18982,4034,2609,82,2369,762,1980
true:Bündnis 90/Die Grünen,2128,86550,11470,970,12996,2988,12968
true:CDU,1925,17765,57615,1045,6997,2321,12846
true:CSU,325,4203,3654,7783,1307,1232,2540
true:Die Linke,1748,17986,7485,387,61502,1337,7958
true:FDP,1230,9742,6215,505,3938,27617,5263
true:SPD,1722,20776,13602,709,8686,2348,58627


# Decision Tree

In [40]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipelineDT = Pipeline([("vect", TfidfVectorizer(ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=10000)),
                     ("clf", DecisionTreeClassifier(random_state = 111))])

In [41]:
modelDT = pipelineDT.fit(X_train, y_train)

In [42]:
vectorizerDT = modelDT.named_steps["vect"]
chiDT = modelDT.named_steps["chi"]
clfDT = modelDT.named_steps["clf"]

In [43]:
predictionDT = modelDT.predict(X_test)

In [44]:
print("******Decision Tree classification report (fulltext, all data)******")
print("accuracy:", accuracy_score(y_test, predictionDT))
print(classification_report(y_test, predictionDT, target_names=target_names, digits=4))

******Decision Tree classification report (fulltext, all data)******
accuracy: 0.4073093171461845
                       precision    recall  f1-score   support

                  AfD     0.4935    0.4009    0.4424     30818
Bündnis 90/Die Grünen     0.3846    0.5214    0.4427    130070
                  CDU     0.3984    0.3711    0.3843    100514
                  CSU     0.2756    0.2425    0.2580     21044
            Die Linke     0.4640    0.3752    0.4149     98403
                  FDP     0.4380    0.3756    0.4044     54510
                  SPD     0.3991    0.3824    0.3906    106470

             accuracy                         0.4073    541829
            macro avg     0.4076    0.3813    0.3910    541829
         weighted avg     0.4118    0.4073    0.4055    541829



In [45]:
print("************************Decision Tree confusion matrix (fulltext, all data)************************")
cm = pd.DataFrame(
    confusion_matrix(y_test, predictionDT, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

************************Decision Tree confusion matrix (fulltext, all data)************************


,pred:AfD,pred:Bündnis 90/Die Grünen,pred:CDU,pred:CSU,pred:Die Linke,pred:FDP,pred:SPD
true:AfD,12354,6656,3389,459,2931,1634,3395
true:Bündnis 90/Die Grünen,3162,67823,16352,3490,13777,7151,18315
true:CDU,2593,25833,37301,3110,9443,5523,16711
true:CSU,478,5699,3418,5103,1739,1457,3150
true:Die Linke,2520,27728,11343,2006,36925,4909,12972
true:FDP,1394,13138,6496,1604,4648,20475,6755
true:SPD,2533,29451,15321,2742,10110,5602,40711


# MLP

In [46]:
#ngram_range = how many words to consider, 1, 2 --> look at every seperate word, and a pair of 2 words: not good --> bad
pipelineMLP = Pipeline([("vect", TfidfVectorizer(ngram_range=(1, 2), stop_words=None, sublinear_tf=True, max_df=0.75, lowercase = False)), 
                     ("chi", SelectKBest(chi2, k=10000)),
                     #("clf", MLPClassifier(max_iter = 6000, random_state = 111))])
                     ("clf", MLPClassifier(hidden_layer_sizes=(7,7), max_iter = 6000, random_state = 111))])

In [47]:
modelMLP = pipelineMLP.fit(X_train, y_train)

In [48]:
vectorizerMLP = modelMLP.named_steps["vect"]
chiMLP = modelMLP.named_steps["chi"]
clfMLP = modelMLP.named_steps["clf"]

In [49]:
predictionMLP = modelMLP.predict(X_test)

In [50]:
print("number of layers:", clfMLP.n_layers_)

number of layers: 4


In [51]:
target_names = clfMLP.classes_

In [52]:
print("******Multilayer Perceptron classification report (fulltext, all data)******")
print("accuracy:", accuracy_score(y_test, predictionMLP))
print(classification_report(y_test, predictionMLP, target_names=target_names, digits=4))

******Multilayer Perceptron classification report (fulltext, all data)******
accuracy: 0.5293773496804343
                       precision    recall  f1-score   support

                  AfD     0.6796    0.5526    0.6095     30818
Bündnis 90/Die Grünen     0.4618    0.6456    0.5385    130070
                  CDU     0.5260    0.4908    0.5078    100514
                  CSU     0.7049    0.2953    0.4162     21044
            Die Linke     0.5886    0.5376    0.5620     98403
                  FDP     0.6932    0.4488    0.5448     54510
                  SPD     0.4937    0.4970    0.4953    106470

             accuracy                         0.5294    541829
            macro avg     0.5925    0.4954    0.5249    541829
         weighted avg     0.5481    0.5294    0.5285    541829



In [53]:
print("***Multilayer Perceptron (two hidden layer, each has 7 neurons) confusion matrix (fulltext, all data)***")
cm = pd.DataFrame(
    confusion_matrix(y_test, predictionMLP, labels=target_names),
    index = ["true:{:}".format(x) for x in target_names],
    columns = ["pred:{:}".format(x) for x in target_names]
)
cm.style.set_properties(**{'text-align': 'center'})

***Multilayer Perceptron (two hidden layer, each has 7 neurons) confusion matrix (fulltext, all data)***


,pred:AfD,pred:Bündnis 90/Die Grünen,pred:CDU,pred:CSU,pred:Die Linke,pred:FDP,pred:SPD
true:AfD,17029,5296,3145,74,2852,846,1576
true:Bündnis 90/Die Grünen,1755,83977,11425,498,12951,3186,16278
true:CDU,1874,23251,49330,794,7062,2102,16101
true:CSU,259,4634,3118,6214,1364,1552,3903
true:Die Linke,1557,24653,7599,283,52905,1049,10357
true:FDP,1239,12992,5639,443,3691,24464,6042
true:SPD,1346,27028,13524,510,9056,2093,52913


# Gradio

In [56]:
!pip install -U -q gradio

In [57]:
import gradio as gr

In [62]:
def makeprediction(tweet):
    realtest = []
    realtest.append(tweet)
    realtest_series = pd.Series(realtest)
    
    #data preparation
    realtest_series = realtest_series.map(remove_URL) #map(lambdax: remove_URL(x))
    realtest_series = realtest_series.map(remove_punct)
    realtest_series = realtest_series.map(remove_stopwords)
    
    
    
    prediction_SVM = model.predict(realtest_series)
    prediction_DT = modelDT.predict(realtest_series)
    prediction_MLP = modelMLP.predict(realtest_series)
    
    
    result = "SVM: " + prediction_SVM + ";  " + "DT: " + prediction_DT + ";  " + "MLP: " + prediction_MLP
    
    return result

In [63]:
iface = gr.Interface(fn=makeprediction, inputs="text", outputs="text", interpretation="default")
iface.launch(share=True)

Running locally at: http://127.0.0.1:7861/
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://28504.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7861/',
 'https://28504.gradio.app')